In [ ]:
from qem.fitting import ImageModelFitting
from scipy.ndimage import gaussian_filter
from qem.crystal_analyzer import CrystalAnalyzer
import hyperspy.api as hs   
import scipy.io as sio
%matplotlib qt

file = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_25.hspy'
s = hs.load(file)
image = s.data
dx =0.071356
matfile_path = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_coords.mat'
matfile = sio.loadmat(matfile_path)
coords = matfile['coords'].astype(float)

image = gaussian_filter(image, 2)
cif_file='/home/zzhang/OneDrive/data/Yang Zhang STO/STEM/72104-CaYAlO4-rotated.cif'
model = ImageModelFitting(image, dx)
model.find_peaks(threshold_rel=0.2, min_distance=10, exclude_border=True)
model.map_lattice(cif_file=cif_file,elements=['Y','Al'],min_distance=20)

In [ ]:
import matplotlib.pyplot as plt
params = model.init_params(guess_radius=False,atom_size=0.9)
prediction = model.predict_local(params)
plt.imshow(prediction)

In [ ]:
# model.import_coordinates(coordinates=coords[:,:2]/dx-2)
# model.import_atom_types = coords[:,2].astype(int)-1
model.fitting_model = 'gaussian'
params = model.init_params(guess_radius=False,atom_size=0.9)

model.fit_global(params,maxiter=3000,tol=1e-6,step_size=1e-3,verbose=True)
# model.fit_random_batch(params,num_epoch=10, batch_size=500,maxiter=50,tol=1e-2,step_size=1e-3,plot=True)

In [ ]:
import numpy as np
plt.hist(model.volume,bins=100)
plt.hist(model.volume[model.atom_types==0],bins=100, alpha=0.5)
plt.hist(model.volume[model.atom_types==1],bins=100, alpha=0.5)
# from scipy.stats import gaussian_kde
# kde = gaussian_kde(model.volume)
# x = np.linspace(min(model.volume), max(model.volume), 1000)
# plt.plot(x, kde(x), color='red')
# kde_Y = gaussian_kde(model.volume[model.atom_types==0])
# plt.plot(x, kde_Y(x), color='blue')
# kde_Al = gaussian_kde(model.volume[model.atom_types==1])
# plt.plot(x, kde_Al(x), color='green')

import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.imshow(model.model)
plt.subplot(1,2,2)
plt.imshow(model.image)